<a href="https://colab.research.google.com/github/kimdonggyu2008/2024_2_Capstone/blob/main/%EC%9A%94%EC%95%BD%EB%AA%A8%EB%8D%B8%EA%B3%BC_%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%85%8B_%ED%95%99%EC%8A%B5_%EC%BD%94%EB%93%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#학습시킬 데이터 로딩 및 임포팅


In [1]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"


In [2]:
# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.2 MB/s eta 0:00:00


In [4]:
from transformers import PegasusTokenizer, PegasusForConditionalGeneration, Trainer, TrainingArguments
from datasets import load_dataset
from google.colab import drive
import torch
import numpy as np
import pickle
import pandas as pd

In [5]:
# # 데이터셋 로드 (Huggingface의 CNN/DailyMail 데이터셋 사용)
# dataset = load_dataset("abisee/cnn_dailymail", "3.0.0")

#데이터 병합 및 토크나이저, 처리(필요시 사용)


In [6]:
# processed_dataset_path = "/content/drive/MyDrive/summarizer/preprcessed/preprocessed_dataset.pkl"
# data_folder_path = "/content/drive/MyDrive/summarizer/data"

In [7]:
# import os
# import pandas as pd
# import torch
# from transformers import PegasusTokenizer, PegasusForConditionalGeneration
# from datasets import Dataset, DatasetDict

# # GPU가 사용 가능한지 확인
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(f"Using device: {device}")

# # 데이터 로드 및 병합
# data_folder_path = "/content/drive/MyDrive/summarizer/data"
# dataframes = []

# for filename in os.listdir(data_folder_path):
#     if filename.endswith(".csv"):
#         file_path = os.path.join(data_folder_path, filename)
#         df = pd.read_csv(file_path, encoding='latin1')

#         # 특정 열 이름만 바꾸기
#         if filename == "news_summary.csv":
#             df = df.rename(columns={
#                 "ctext": "article",  # 'ctext'를 'article'로 변경
#                 "text": "highlights"  # 'text'를 'highlights'로 변경
#             })

#         elif filename == "news_summary_more.csv":
#             df = df.rename(columns={
#                 "headlines": "highlights",  # 'headlines'를 'highlights'로 변경
#                 "text": "article"  # 'text'를 'article'로 변경
#             })

#         # 데이터프레임 리스트에 추가
#         dataframes.append(df)

# # 데이터프레임 병합
# combined_df = pd.concat(dataframes, ignore_index=True)

# # NaN을 빈 문자열로 대체
# combined_df['article'] = combined_df['article'].fillna("")

# # 'article' 열의 값이 문자열인지 확인하고, 문자열이 아닌 값은 빈 문자열로 대체
# combined_df['article'] = combined_df['article'].apply(lambda x: str(x) if isinstance(x, str) else "")

# # 빈 행(빈 문자열 포함)을 삭제
# combined_df = combined_df[combined_df['article'].str.strip() != ""]

# #combined_df.to_csv("/content/drive/MyDrive/summarizer/combined_dataset.csv", index=False)


# dataset = Dataset.from_pandas(combined_df)

# train_test = dataset.train_test_split(test_size=0.1)
# datasets = DatasetDict({
#     'train': train_test['train'],
#     'test': train_test['test']
# })



# # Pegasus 모델 및 토크나이저 준비
# model_name = "google/pegasus-xsum"
# tokenizer = PegasusTokenizer.from_pretrained(model_name)
# model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)

# # 전처리 함수 정의 (GPU에서 입력 데이터를 처리하도록 변경)
# def preprocess_function(examples):
#     inputs = tokenizer(examples['article'], padding="max_length", truncation=True)
#     labels = tokenizer(examples['highlights'], padding="max_length", truncation=True)
#     inputs["labels"] = labels["input_ids"]
#     return inputs

# # 데이터셋을 토크나이징, 데이터 있으면 안해도 됨
# tokenized_datasets = datasets.map(preprocess_function, batched=True)



In [8]:
# with open("/content/drive/MyDrive/summarizer/preprocessed/tokenized_datasets.pkl", "wb") as f:
#     pickle.dump(tokenized_datasets, f)

#데이터 로딩


In [10]:
import pickle

preprocessed_data_path = "/content/drive/MyDrive/summarizer/preprcessed/preprocessed_dataset.pkl"
dataset_df = pd.read_pickle(preprocessed_data_path)


In [ ]:
# Load tokenized dataset
with open('/content/drive/MyDrive/summarizer/preprcessed/tokenized_datasets.pkl', 'rb') as f:
    tokenized_datasets = pickle.load(f)


#학습 및 저장


In [11]:
# 모델과 데이터셋 설정
model_name = "google/pegasus-xsum"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/87.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.52M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/259 [00:00<?, ?B/s]

In [12]:
from transformers import PegasusTokenizer, PegasusForConditionalGeneration, Trainer, TrainingArguments, EarlyStoppingCallback

In [13]:
model_checkpoint_path = "/content/drive/MyDrive/summarizer/checkpoints"

In [14]:
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Colab Notebooks/Pegasus_요약모델/results",
    evaluation_strategy="epoch",  # 매 에포크마다 평가
    save_strategy="epoch",  # 매 에포크마다 체크포인트 저장
    learning_rate=2e-5,
    per_device_train_batch_size=2,  # 배치 크기 설정
    per_device_eval_batch_size=2,    # 배치 크기 설정
    num_train_epochs=10,  # 에포크 수
    weight_decay=0.01,
    logging_dir="/content/drive/MyDrive/Colab Notebooks/Pegasus_요약모델/logs",
    logging_steps=10,
    save_total_limit=2,  # 최대 2개의 체크포인트만 저장
    load_best_model_at_end=True,  # 가장 성능이 좋은 모델 로드
    metric_for_best_model="eval_loss",  # 가장 좋은 모델을 선택할 기준
    greater_is_better=False  # 낮은 eval_loss가 더 좋은 모델임을 나타냄
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [15]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]  # 조기 종료: 성능 개선 없으면 3번 후 종료
)

In [16]:
trainer.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
# final_model_path = "/content/drive/MyDrive/summarizer/final_model"
# trainer.save_model(final_model_path)
# print("Final model saved successfully.")
model.save_pretrained("/content/drive/MyDrive/summarizer/final_model")
tokenizer.save_pretrained("/content/drive/MyDrive/summarizer/final_model")

In [ ]:
from transformers import T5ForConditionalGeneration
model = T5ForConditionalGeneration.from_pretrained(final_model_path)
print("Final model loaded successfully.")

#테스트 요약

In [ ]:
# 학습 시작 및 요약 결과 확인
def summarize_article(article_text):
    inputs = tokenizer(article_text, truncation=True, padding="longest", return_tensors="pt")
    summary_ids = model.generate(**inputs, max_length=60, num_beams=5, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary


In [ ]:
# 사용자 입력 문장을 요약
article_text = "여기다가 입력"
summary = summarize_article(article_text)
print("\nSummary:")
print(summary)